In [1]:
from datasets import load_dataset
ds = load_dataset("simplescaling/s1K-1.1")["train"]
ds[0]

{'solution': '128',
 'question': 'Given a rational number, write it as a fraction in lowest terms and calculate the product of the resulting numerator and denominator. For how many rational numbers between 0 and 1 will $20_{}^{}!$ be the resulting product?',
 'cot_type': 'math',
 'source_type': 'qq8933/AIME_1983_2024',
 'metadata': "{'ID': '1991-5', 'Year': 1991, 'Problem Number': 5, 'Part': None}",
 'gemini_thinking_trajectory': '\nThe problem asks for the number of rational numbers between 0 and 1 such that when the rational number is written as a fraction in lowest terms, the product of the numerator and the denominator is $20!$.\n\nLet the rational number be $r$, where $0 < r < 1$.\nWe can write $r$ as a fraction $\\frac{a}{b}$, where $a$ and $b$ are positive integers, and $\\gcd(a, b) = 1$.\nSince $0 < r < 1$, we have $0 < \\frac{a}{b} < 1$, which implies $0 < a < b$.\n\nThe product of the numerator and the denominator of the fraction in lowest terms is $a \\times b$.\nWe are give

In [2]:
ds_format = load_dataset('Asap7772/aime_2025_responses_qwen3-1.7b', split='train')
print(ds_format[0]['response'][0])

<think>
Okay, so I need to find all integer bases b greater than 9 where the number 17_b is a divisor of 97_b. Then sum all those bases. Let me start by understanding what the problem is asking.

First, bases in numeral systems. In base b, the number 17_b represents 1*b^1 + 7*b^0 = b + 7. Similarly, 97_b is 9*b + 7. So the problem is asking for bases b > 9 where (b + 7) divides (9b + 7). In other words, (b + 7) divides (9b + 7). 

So, mathematically, we need that (9b + 7) mod (b + 7) = 0. Let me recall how to handle divisibility in such cases. One method is to perform polynomial division or use the remainder theorem. 

Alternatively, since we have a divisor (b + 7) and the dividend (9b + 7), we can write:

Let me think. If (b + 7) divides (9b + 7), then there exists some integer k such that 9b + 7 = k*(b + 7). 

So, solving for k:

9b + 7 = k*b + 7k

Let me rearrange terms:

9b - k*b = 7k - 7

b*(9 - k) = 7(k - 1)

Therefore, b = [7(k - 1)] / (9 - k)

But since b must be an integer gre

In [3]:
from textwrap import dedent
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-1.7B')
math_prefix = "Solve the following math problem. Give your final answer as \\boxed{}."
apply_prefix = lambda x: f'{math_prefix}\n{x}'
thinking_template = """
<think>
{thinking_str}
</think>

{response}
"""
thinking_template = dedent(thinking_template).strip()
print(thinking_template)

<think>
{thinking_str}
</think>

{response}


In [4]:
import os
def map_fn_gemini(example):
    current_problem = example['question']
    current_thinking = example['gemini_thinking_trajectory']
    current_response = example['gemini_attempt']
    
    example['query'] = tokenizer.apply_chat_template([{'role': 'user', 'content': apply_prefix(current_problem)}], add_generation_prompt=True, tokenize=False)
    example['completion'] = thinking_template.format(thinking_str=current_thinking, response=current_response)
    return example

ds_gemini = ds.map(map_fn_gemini, num_proc=os.cpu_count(), remove_columns=ds.column_names)
print(ds_gemini[0]['query'])
print(ds_gemini[0]['completion'])


<|im_start|>user
Solve the following math problem. Give your final answer as \boxed{}.
Given a rational number, write it as a fraction in lowest terms and calculate the product of the resulting numerator and denominator. For how many rational numbers between 0 and 1 will $20_{}^{}!$ be the resulting product?<|im_end|>
<|im_start|>assistant

<think>

The problem asks for the number of rational numbers between 0 and 1 such that when the rational number is written as a fraction in lowest terms, the product of the numerator and the denominator is $20!$.

Let the rational number be $r$, where $0 < r < 1$.
We can write $r$ as a fraction $\frac{a}{b}$, where $a$ and $b$ are positive integers, and $\gcd(a, b) = 1$.
Since $0 < r < 1$, we have $0 < \frac{a}{b} < 1$, which implies $0 < a < b$.

The product of the numerator and the denominator of the fraction in lowest terms is $a \times b$.
We are given that $a \times b = 20!$.

We need to find the number of pairs $(a, b)$ such that:
1. $a$ and $

In [5]:
ds_gemini = ds_gemini.train_test_split(test_size=0.05, seed=42)
os.makedirs('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_gemini_sft', exist_ok=True)
ds_gemini['train'].to_parquet('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_gemini_sft/train.parquet')
ds_gemini['test'].to_parquet('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_gemini_sft/test.parquet')
ds_gemini.push_to_hub('Asap7772/s1K-1.1-gemini-qwensft')

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/Asap7772/s1K-1.1-gemini-qwensft/commit/7f70a8048ed69cee050bb371eef84d7b9b171a53', commit_message='Upload dataset', commit_description='', oid='7f70a8048ed69cee050bb371eef84d7b9b171a53', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Asap7772/s1K-1.1-gemini-qwensft', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Asap7772/s1K-1.1-gemini-qwensft'), pr_revision=None, pr_num=None)

In [6]:
import os
def map_fn_deepseek(example):
    current_problem = example['question']
    current_thinking = example['deepseek_thinking_trajectory']
    current_response = example['deepseek_attempt']
    
    example['query'] = tokenizer.apply_chat_template([{'role': 'user', 'content': apply_prefix(current_problem)}], add_generation_prompt=True, tokenize=False)
    example['completion'] = thinking_template.format(thinking_str=current_thinking, response=current_response)
    return example

ds_deepseek = ds.map(map_fn_deepseek, num_proc=os.cpu_count(), remove_columns=ds.column_names)
print(ds_deepseek[0]['query'])
print(ds_deepseek[0]['completion'])


Map (num_proc=24):   0%|          | 0/1000 [00:00<?, ? examples/s]

<|im_start|>user
Solve the following math problem. Give your final answer as \boxed{}.
Given a rational number, write it as a fraction in lowest terms and calculate the product of the resulting numerator and denominator. For how many rational numbers between 0 and 1 will $20_{}^{}!$ be the resulting product?<|im_end|>
<|im_start|>assistant

<think>
Alright, so I need to figure out how many rational numbers between 0 and 1 have a product of the numerator and denominator (when written in lowest terms) equal to 20 factorial. Let me start by understanding the problem.

First, a rational number between 0 and 1 can be written as a fraction \(\frac{a}{b}\) where \(0 < a < b\) and \(a\) and \(b\) are coprime positive integers. The product \(a \times b\) is said to be equal to 20! which is a huge number. So I need to find all pairs \((a, b)\) such that \(a \times b = 20!\), \(0 < a < b\), and \(\gcd(a, b) = 1\). Then count how many such pairs exist.

Let me break down the problem.

Given that \

In [7]:
ds_deepseek = ds_deepseek.train_test_split(test_size=0.05, seed=42)
os.makedirs('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_deepseek_sft', exist_ok=True)
ds_deepseek['train'].to_parquet('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_deepseek_sft/train.parquet')
ds_deepseek['test'].to_parquet('/home/anikait.singh/rl_behaviors_verl_stable/data_s1_deepseek_sft/test.parquet')
ds_deepseek.push_to_hub('Asap7772/s1K-1.1-deepseek-qwensft')

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Asap7772/s1K-1.1-deepseek-qwensft/commit/83e1900f4c7dd0f545070855d5b4f39a7df01bf3', commit_message='Upload dataset', commit_description='', oid='83e1900f4c7dd0f545070855d5b4f39a7df01bf3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Asap7772/s1K-1.1-deepseek-qwensft', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Asap7772/s1K-1.1-deepseek-qwensft'), pr_revision=None, pr_num=None)